### Continuous mode
The image below illustrate how the continuous mode works

![Continuous mode](./img/Serial_Communication_Continuous_Mode.png)

Basically, we will send the commands to the device and whenever we collect new data, we will print it to the console, but also put in a dictionary, that we will later save into a JSON file. In this mode, we will repeat these steps in an endless loop, so we will send information back and forth in the serial without the need to get input from the user. All the information will be stored in the file, and we will be able to use this data for analysis.
This mode has the advantage that we can collect data from the device without the need to send a command to the device every time we want to read the sensors. This is useful when we want to collect data from the device for a long period of time.

### Example code
Here's the complete example code. Note that we have used what´s called ASCII art to write down a beautiful welcome message. Command interfaces do not need to be boring!



In [ ]:
simulation_mode = True # Set this variable to True to simulate the data
if port is not None or simulation_mode:
    print(" _      __    __")
    print("| | /| / /__ / /______  __ _  ___")
    print("| |/ |/ / -_) / __/ _ \\/  ' \\/ -_)")
    print("|__/|__/\\__/_/\\__/\\___/_/_/_/\\__/")

    print("Welcome to the Arduino control panel")
    print("You can use the following commands:")
    print("1. Read humidity")
    print("2. Read temperature")
    print("3. Read humidity and temperature")
    print("4. Read soil moisture")
    print("5. read all in continuous mode")
    print("Press Ctrl+C to exit")
    while True:

        command = input("Enter command: ")
        if command in ['1', '2', '3', '4'] and not simulation_mode:
            signal = command.encode('utf-8') # Convert the command to a binary string
            arduino.write(signal) # Send the command to the device
            arduino.flush() # Wait until the command is sent
            raw_data = arduino.readline() # Read the data from the device
            print(raw_data) # Print the response from the device
        elif command == '5':
            print("Entering continuous mode. Press Ctrl+C to exit")
            while True:
                time.sleep(1) # Wait for 1 second
                # Let´s put the data in a dictionary.
                data = {"timestamp": time.strftime("%Y-%m-%d %H:%M:%S")}

                if simulation_mode: # If we are in testing mode, we will simulate the data
                    data["humidity"] = random.randint(0, 100)
                    data["temperature"] = random.randint(0, 100)
                    data["soil_moisture"] = random.randint(0, 1000)
                else: # If we are not in testing mode, we will read the data from the device
                    # First send a signal to read humidity and temperature
                    signal = b'3'
                    arduino.write(signal) # Send the command to the device
                    arduino.flush() # Wait until the command is sent
                    raw_data = arduino.readline() # Read the data from the device

                    #Incoming data is in the format b'Humidity: 50.00 % Temperature: 23.00 \n'
                    # We need to split the string into a list of strings
                    raw_data = raw_data.decode('utf-8').strip().split(' ')
                    # Now we need to convert the strings to floats and add them to the dictionary
                    data["humidity"] = float(raw_data[1])
                    data["temperature"] = float(raw_data[4])
                    time.sleep(1) # Wait for 1 second
                    # Now send a signal to read soil moisture
                    signal = b'4'
                    arduino.write(signal) # Send the command to the device
                    arduino.flush() # Wait until the command is sent
                    raw_data = arduino.readline() # Read the data from the device
                    # Incoming data is in the format b'Soil Moisture: 350 \n'
                    # Decode the data and split it into a list of strings
                    raw_data = raw_data.decode('utf-8').strip().split(' ')
                    # Get the soil moisture value and store it in the dictionary
                    data["soil_moisture"] = float(raw_data[2])

                print(data)
                # Now we can save incoming data into the file. We need to open the file in append mode, and check whether the file contains previous data
                with open("data.csv", "a+") as f:
                    # First we need to check whether the file contains previous data
                    f.seek(0) # Move the cursor to the beginning of the file
                    previous_data = f.read() # Read the file
                    if previous_data == "": # If the file is empty, we need to add the header
                        f.write("timestamp,humidity,temperature,soil_moisture\n")
                        # Now we can write the data to the file
                        f.write(f"{data['timestamp']},{data['humidity']},{data['temperature']},{data['soil_moisture']}\n")
                    else: # If the file is not empty, we just need to move the cursor to the end of the file and add the data
                        f.seek(0, 2) # Move the cursor to the end of the file
                        f.write(f"{data['timestamp']},{data['humidity']},{data['temperature']},{data['soil_moisture']}\n")
        else:
            print("Invalid command")

And this is it! We have successfully established a serial connection with the Arduino board and we have used it to control the device. We have also saved the data in a file!
In the next section, we provide a different example based on a divide and conquer! strategy. We will have two different python scripts, one to get the input from the user and send it to an intermediate file named ````command.csv````, and another one that continously reads the data from the device and the command file and sends the data to a file named ````data.csv````. This is a more complex example, but it is more robust and it allows enhanced control and GUI development.



### Interact with the device using two scripts
In this section, we will use two different scripts to interact with the device. The first script will be used to manage user interaction, collecting the user commands, and sending them to a file which will have the commands entered by the user. The second will actually interact with the arduino device using the serial port. This is a more complex example, but it is more robust and it allows enhanced control and GUI development. Here is an illustration of how it works:

![Arduino serial communication](img/Serial_Communication_new_stack.png)


Use this code as a template to build your script to manage user interaction:

```python
import time
# Let´s define a list of dictionaries for our supported commands:

# TODO: Change this to your actual actuator commands
commands = [
    {"user_command": "1", "description": "send actuator command"},
    {"user_command": "2", "description": "send another actuator command"}
]

# Let´s print the list of commands
print("Welcome to the Arduino control panel")
print("You can use the following commands:")
for command in commands:
    print(f"{command['user_command']}. {command['description']}")

print("Press Ctrl+C to exit")

while True:
    command = input("Enter command: ")
    # Let´s check whether the command is valid
    valid_command = False
    for c in commands:
        if command == c["user_command"]:
            valid_command = True
            break
    if valid_command:
        # Let´s write the command to the file
        with open("command.csv", "a") as f:
            current_time = time.strftime("%Y-%m-%d %H:%M:%S")
            f.write(f"{current_time}, {command}")
    else:
        print("Invalid command")
```

With this script, we will be able to collect the user commands and write them to a file named ````command.csv````. Since, we are opening the file in write mode ```"a"``` this file will have at most one command  Now, we need to write the script that will read the commands from the file and interact with the device. This is the template script:

```python
import serial
import time
import random

port = None # Initialize the port variable. If you already know the serial port where the Arduino board is connected, you can assign it to this variable, replacing with the line below
#port = 'COM7'
if port is not None:
    ports = serial.tools.list_ports.comports()
    for p in ports:
        #print(f"{p.device} {p.manufacturer}") # Uncomment this line to print the list of serial ports
        if p.manufacturer is not None and 'Arduino' in p.manufacturer:
            port = p.device
            break

simulation_mode = True # Set this variable to True to simulate the data

while True:
# Let´s collect data from the device
    if port is not None or simulation_mode:
        command = None # Initialize the command variable
        # Let´s sleep for one second
        time.sleep(1)
        # Let´s open the file to read the commands
        with open("command.csv", "r") as f:
            # Let´s read the last line of the file
            last_line = f.readlines()[-1]
            # Let´s check whether the file contains a command
            if last_line:
                # Let´s split the line into a list of strings
                last_line = last_line.split(",")
                # Let´s get the command
                command = last_line[1]


        # Now we can send the command to the device
        if command == '1':
            print("Sending command 1")
            # Let´s send the command to the device
            signal = b'1'
            arduino.write(signal) # Send the command to the device
            arduino.flush() # Wait until the command is sent
            raw_data = arduino.readline() # Read the data from the device
            print(raw_data) # Print the response from the device
        elif command == '2':
            print("Sending command 2")
            # Let´s send the command to the device
            signal = b'2'
            arduino.write(signal) # Send the command to the device
            arduino.flush() # Wait until the command is sent
            raw_data = arduino.readline() # Read the data from the device
            print(raw_data) # Print the response from the device

        # TODO: Modify this to collect the data from your device
        # Let´s collect data from the device
        if simulation_mode: # If we are in testing mode, we will simulate the data
            data["humidity"] = random.randint(0, 100)
            data["temperature"] = random.randint(0, 100)
            data["soil_moisture"] = random.randint(0, 1000)
        else: # If we are not in testing mode, we will read the data from the device
            # First send a signal to read humidity and temperature
            signal = b'3'
            arduino.write(signal) # Send the command to the device
            arduino.flush() # Wait until the command is sent
            raw_data = arduino.readline() # Read the data from the device

            #Incoming data is in the format b'Humidity: 50.00 % Temperature: 23.00 \n'
            # We need to split the string into a list of strings
            raw_data = raw_data.decode('utf-8').strip().split(' ')
            # Now we need to convert the strings to floats and add them to the dictionary
            data["humidity"] = float(raw_data[1])
            data["temperature"] = float(raw_data[4])
            time.sleep(1) # Wait for 1 second
            # Now send a signal to read soil moisture
            signal = b'4'
            arduino.write(signal) # Send the command to the device
            arduino.flush() # Wait until the command is sent
            raw_data = arduino.readline() # Read the data from the device
            # Incoming data is in the format b'Soil Moisture: 350 \n'
            # Decode the data and split it into a list of strings
            raw_data = raw_data.decode('utf-8').strip().split(' ')
            # Get the soil moisture value and store it in the dictionary
            data["soil_moisture"] = float(raw_data[2])

        print(data)
        # Now we can save incoming data into the file. We need to open the file in append mode, and check whether the file contains previous data

        with open("data.csv", "a+") as f:
            # First we need to check whether the file contains previous data
            f.seek(0) # Move the cursor to the beginning of the file
            previous_data = f.read() # Read the file
            if previous_data == "": # If the file is empty, we need to add the header
                f.write("timestamp,humidity,temperature,soil_moisture\n")
                # Now we can write the data to the file
                f.write(f"{data['timestamp']},{data['humidity']},{data['temperature']},{data['soil_moisture']}\n")
            else: # If the file is not empty, we just need to move the cursor to the end of the file and add the data
                f.seek(0, 2) # Move the cursor to the end of the file
                f.write(f"{data['timestamp']},{data['humidity']},{data['temperature']},{data['soil_moisture']}\n")
```
